# WikiPathways SPARQL queries

In [13]:
import sys

!{sys.executable} -m pip install --upgrade pip 
!{sys.executable} -m pip install watermark

try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except ImportError:
    !{sys.executable} -m pip install sparqlwrapper
    from SPARQLWrapper import SPARQLWrapper, JSON

try:
    import pandas as pd
except ImportError:
    !{sys.executable} -m pip install pandas
    import pandas as pd

try:
    import time
except ImportError:
    !{sys.executable} -m pip install time
    import time
pd.set_option('display.max_colwidth', -1)

Requirement already up-to-date: pip in /home/marvin.martens/anaconda3/lib/python3.7/site-packages (20.2.3)


In [14]:
SPARQL = SPARQLWrapper("http://localhost:8890/sparql/")
SPARQL.setReturnFormat(JSON) 

In [38]:
start = time.time()
listofdata = []

#Pathways

data = pd.DataFrame(columns=['item','number'])
sparqlquery = '''SELECT DISTINCT count(distinct ?pathway) as ?pathwayCount
WHERE {
  ?pathway a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["pathwayCount"]["value"],
            'item' : 'allPathways',
        }, ignore_index=True)
    listofdata.append(result["pathwayCount"]["value"])

sparqlquery = '''SELECT DISTINCT count(distinct ?pathway) as ?pathwayCount
WHERE {
  ?pathway a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["pathwayCount"]["value"],
            'item' : 'allCuratedPathways',
        }, ignore_index=True)
    listofdata.append(result["pathwayCount"]["value"])

sparqlquery = '''SELECT DISTINCT count(distinct ?pathway) as ?pathwayCount
WHERE {
  ?pathway a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:Reactome_Approved)
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["pathwayCount"]["value"],
            'item' : 'allReactomePathways',
        }, ignore_index=True)
    listofdata.append(result["pathwayCount"]["value"])

#Genes

sparqlquery = '''SELECT DISTINCT count(?ensId) as ?Ensembl 
WHERE {
  ?dataNodes a wp:DataNode ;
  wp:bdbEnsembl ?ensId ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["Ensembl"]["value"],
            'item' : 'allGenes',
        }, ignore_index=True)
    listofdata.append(result["Ensembl"]["value"])

sparqlquery = '''SELECT DISTINCT count(distinct ?ensId) as ?Ensembl 
WHERE {
  ?dataNodes a wp:DataNode ;
  wp:bdbEnsembl ?ensId ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["Ensembl"]["value"],
            'item' : 'UniqueGenes',
        }, ignore_index=True)
    listofdata.append(result["Ensembl"]["value"])

sparqlquery = '''SELECT DISTINCT count(?ensId) as ?Ensembl 
WHERE {
  ?dataNodes a wp:DataNode ;
  wp:bdbEnsembl ?ensId ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["Ensembl"]["value"],
            'item' : 'allGenesCuratedPathways',
        }, ignore_index=True)
    listofdata.append(result["Ensembl"]["value"])
sparqlquery = '''SELECT DISTINCT count(distinct ?ensId) as ?Ensembl 
WHERE {
  ?dataNodes a wp:DataNode ;
  wp:bdbEnsembl ?ensId ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["Ensembl"]["value"],
            'item' : 'UniqueGenesCuratedPathways',
        }, ignore_index=True)
    listofdata.append(result["Ensembl"]["value"])

#PubMed

sparqlquery = '''SELECT  count(?pubmed) as ?pubmedCount
WHERE {
  ?pubmed a wp:PublicationReference ;
  dcterms:isPartOf ?pathway.
  ?pathway a wp:Pathway;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["pubmedCount"]["value"],
            'item' : 'allPublications',
        }, ignore_index=True)
    listofdata.append(result["pubmedCount"]["value"])

sparqlquery = '''SELECT count(distinct ?pubmed) as ?pubmedCount 
WHERE {
  ?pubmed a wp:PublicationReference ;
  dcterms:isPartOf ?pathway.
  ?pathway a wp:Pathway;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["pubmedCount"]["value"],
            'item' : 'UniquePublications',
        }, ignore_index=True)
    listofdata.append(result["pubmedCount"]["value"])

sparqlquery = '''SELECT DISTINCT count(?pubmed) as ?pubmedCount
WHERE {
  ?pubmed a wp:PublicationReference ;
  dcterms:isPartOf ?pathway.
  ?pathway a wp:Pathway;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["pubmedCount"]["value"],
            'item' : 'allPublicationsCuratedPathways',
        }, ignore_index=True)
    listofdata.append(result["pubmedCount"]["value"])
sparqlquery = '''SELECT DISTINCT count(distinct ?pubmed) as ?pubmedCount 
WHERE {
  ?pubmed a wp:PublicationReference ;
  dcterms:isPartOf ?pathway.
  ?pathway a wp:Pathway;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["pubmedCount"]["value"],
            'item' : 'UniquePublicationsCuratedPathways',
        }, ignore_index=True)
    listofdata.append(result["pubmedCount"]["value"])

#Metabolites

sparqlquery = '''SELECT count(?metabolite) as ?metaboliteCount 
WHERE {
  ?metabolite a wp:Metabolite ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    x = result["metaboliteCount"]["value"]

sparqlquery = '''SELECT count(?metabolite) as ?metaboliteCount 
WHERE {
  ?metabolite a wp:Metabolite ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
  
  OPTIONAL { ?metabolite wp:bdbWikidata ?wikidata . } 
  OPTIONAL { ?metabolite wp:bdbChEBI ?chebi . }  
  OPTIONAL { ?metabolite wp:bdbHmdb ?hmdb . }  
  OPTIONAL { ?metabolite wp:bdbChemspider ?chemspider . }  
  OPTIONAL { ?metabolite wp:bdbPubChem ?pubchem . }  

  FILTER (!BOUND(?wikidata))  
  FILTER (!BOUND(?chebi))  
  FILTER (!BOUND(?hmdb))  
  FILTER (!BOUND(?chemspider)) 
  FILTER (!BOUND(?pubchem)) 
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    y = result["metaboliteCount"]["value"]

data = data.append({
            'number' : str(int(int(x)-int(y))),
            'item' : 'allMetabolites',
        }, ignore_index=True)
listofdata.append(str(int(int(x)-int(y))))
sparqlquery = '''SELECT count(distinct ?metabolite) as ?metaboliteCount  
WHERE {
  ?metabolite a wp:Metabolite ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    x = result["metaboliteCount"]["value"]

sparqlquery = '''SELECT count(distinct ?metabolite) as ?metaboliteCount  
WHERE {
  ?metabolite a wp:Metabolite ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
  
  OPTIONAL { ?metabolite wp:bdbWikidata ?wikidata . }  
  OPTIONAL { ?metabolite wp:bdbChEBI ?chebi . }  
  OPTIONAL { ?metabolite wp:bdbHmdb ?hmdb . }  
  OPTIONAL { ?metabolite wp:bdbChemspider ?chemspider . }  
  OPTIONAL { ?metabolite wp:bdbPubChem ?pubchem . } 

  FILTER (!BOUND(?wikidata))  
  FILTER (!BOUND(?chebi)) 
  FILTER (!BOUND(?hmdb))  
  FILTER (!BOUND(?chemspider))
  FILTER (!BOUND(?pubchem)) 
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    y = result["metaboliteCount"]["value"]

data = data.append({
            'number' : str(int(int(x)-int(y))),
            'item' : 'UniqueMetabolites',
        }, ignore_index=True)
listofdata.append(str(int(int(x)-int(y))))
sparqlquery = '''SELECT count(?metabolite) as ?metaboliteCount  
WHERE {
  ?metabolite a wp:Metabolite ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    x = result["metaboliteCount"]["value"]

sparqlquery = '''SELECT count(?metabolite) as ?metaboliteCount  
WHERE {
  ?metabolite a wp:Metabolite ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
  OPTIONAL { ?metabolite wp:bdbWikidata ?wikidata . }  
  OPTIONAL { ?metabolite wp:bdbChEBI ?chebi . } 
  OPTIONAL { ?metabolite wp:bdbHmdb ?hmdb . }  
  OPTIONAL { ?metabolite wp:bdbChemspider ?chemspider . }  
  OPTIONAL { ?metabolite wp:bdbPubChem ?pubchem . }  

  FILTER (!BOUND(?wikidata))  
  FILTER (!BOUND(?chebi))  
  FILTER (!BOUND(?hmdb))  
  FILTER (!BOUND(?chemspider)) 
  FILTER (!BOUND(?pubchem))  
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    y = result["metaboliteCount"]["value"]

data = data.append({
            'number' : str(int(int(x)-int(y))),
            'item' : 'allMetabolitesCuratedPathways',
        }, ignore_index=True)
listofdata.append(str(int(int(x)-int(y))))
sparqlquery = '''SELECT count(distinct ?metabolite) as ?metaboliteCount  
WHERE {
  ?metabolite a wp:Metabolite ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    x = result["metaboliteCount"]["value"]

sparqlquery = '''SELECT count(distinct ?metabolite) as ?metaboliteCount 
WHERE {
  ?metabolite a wp:Metabolite ;
  dcterms:isPartOf ?pw.
  ?pw a wp:Pathway;
  dc:title ?title ;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
  OPTIONAL { ?metabolite wp:bdbWikidata ?wikidata . }  
  OPTIONAL { ?metabolite wp:bdbChEBI ?chebi . }  
  OPTIONAL { ?metabolite wp:bdbHmdb ?hmdb . } 
  OPTIONAL { ?metabolite wp:bdbChemspider ?chemspider . }  
  OPTIONAL { ?metabolite wp:bdbPubChem ?pubchem . } 

  FILTER (!BOUND(?wikidata)) 
  FILTER (!BOUND(?chebi)) 
  FILTER (!BOUND(?hmdb)) 
  FILTER (!BOUND(?chemspider))  
  FILTER (!BOUND(?pubchem)) 
}'''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    y = result["metaboliteCount"]["value"]

data = data.append({
            'number' : str(int(int(x)-int(y))),
            'item' : 'UniqueMetabolitesCuratedPathways',
        }, ignore_index=True)
listofdata.append(str(int(int(x)-int(y))))

#Interactions
sparqlquery = '''SELECT count( ?interaction) as ?countInteraction
WHERE {
  ?interaction a wp:Interaction ;
  dcterms:isPartOf ?pathway.
  ?pathway a wp:Pathway;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string .
} '''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["countInteraction"]["value"],
            'item' : 'allInteractions',
        }, ignore_index=True)
    listofdata.append(result["countInteraction"]["value"])

sparqlquery = '''SELECT count( ?interaction) as ?countInteraction
WHERE {
  ?interaction a wp:Interaction ;
  dcterms:isPartOf ?pathway.
  ?pathway a wp:Pathway;
  wp:organism ?organism ;
  wp:organismName "Homo sapiens"^^xsd:string ;
  wp:ontologyTag ?pwo.
  FILTER (?pwo = cur:AnalysisCollection)
} '''
SPARQL.setQuery(sparqlquery)
results = SPARQL.query().convert()

for result in results["results"]["bindings"]:
    data = data.append({
            'number' : result["countInteraction"]["value"],
            'item' : 'allInteractionsCuratedPathways',
        }, ignore_index=True)
    listofdata.append(result["countInteraction"]["value"])

display(data)
print(listofdata)
end = time.time()
print(end - start)

,item,number
0,allPathways,853
1,allCuratedPathways,405
2,allReactomePathways,448
3,allGenes,44274
4,UniqueGenes,11523
5,allGenesCuratedPathways,16941
6,UniqueGenesCuratedPathways,5839
7,allPublications,28500
8,UniquePublications,20097
9,allPublicationsCuratedPathways,3421


['853', '405', '448', '44274', '11523', '16941', '5839', '28500', '20097', '3421', '2896', '8948', '3368', '2881', '1842', '40699', '13369']
0.9404137134552002
